In [1]:
import os
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

os.environ["SPARK_HOME"] = "/usr/local/spark"
os.environ["PYSPARK_PYTHON"] = "/home/pigidser/anaconda3/bin/python3"
os.environ["PYSPARK_DRIVER_PYTHON"] = "python3"
os.environ["PYSPARK_SUBMIT_ARGS"] = "pyspark-shell"

spark = SparkSession.builder.master("local").appName("spark_test").getOrCreate()

In [2]:
# Создание RDD
# Давайте создадим простейший RDD - слова файла (как строки)

wrds = spark.sparkContext.parallelize("this is spark and it is great".split(),3) # 3 - number of partitions

In [4]:
# Из файла
# Прочитаем файл и создадим RDD со строками файла

lines = spark.sparkContext.textFile("countries_of_the_world.csv")
lines.first()

'Country,Region,Population,Area (sq. mi.),Pop. Density (per sq. mi.),Coastline (coast/area ratio),Net migration,Infant mortality (per 1000 births),GDP ($ per capita),Literacy (%),Phones (per 1000),Arable (%),Crops (%),Other (%),Climate,Birthrate,Deathrate,Agriculture,Industry,Service'

In [5]:
# Трансформации
# Начнем с фильтрации, давайте сделаем что-то со словами, ниже - разберем, что получилось
# и почему RDD не эффективны

def startsWithS(individual):
    return individual.startswith("s")

wrds.filter(lambda word: startsWithS(word)).collect()

['spark']

Что произошло:

- мы написали функцию на python (startsWithS)
- она была spark-ом как-то переведена на java (executor-ы "понимают" только java)
- на каждом узле кластера (где "живут" разделы нашего RDD) для каждого элемента раздела RDD была вызвана эта функция
- на вход был подан "переведенный" в python строку (десериализация) элемент раздела RDD с этого узла
- функция отработала, вернула True или False
- в раздел нового RDD попала часть элементов старого (возможно, произошла еще одна сериализация)
- все разделы собираются на драйвер (collect) и происходит еще одна десериализация (в типы python)

Обратите внимание - результат collect() - список строк (чисто python объект), т.е. никакого преобразования в объекты spark не происходит. В функции startsWithS() мы работаем со строками. Spark делает за нас все преобразования, но это снижает эффективность, нужно иметь это в виду

In [6]:
# dictinct()
# У нас есть повторяющиеся слова - удалим их

wrds.distinct().collect()

['it', 'is', 'and', 'this', 'spark', 'great']

In [7]:
# map()
# Преобразуем наши слова в пары - слово, длина

def getLen(individual):
    return (individual,len(individual))

wrds.map(getLen).collect()

[('this', 4),
 ('is', 2),
 ('spark', 5),
 ('and', 3),
 ('it', 2),
 ('is', 2),
 ('great', 5)]

In [8]:
# flatMap()
# Разобъем слова по символам (т.е. увеличим количество разделов - каждый символ = раздел)
# и соберем на driver первые 6 разделов

def toChars(individual):
    return list(individual)

wrds.flatMap(toChars).take(6)

['t', 'h', 'i', 's', 'i', 's']

In [9]:
# sort()
# Давайте упорядочим слова по их длине (по убыванию)

wrds.sortBy(lambda word: len(word) * -1).collect()

['spark', 'great', 'this', 'and', 'is', 'it', 'is']

In [10]:
# Действия
# мы уже видели collect()/take(), давайте посмотрим reduce(): оставим самое длинное слово из нашего набора

def wordLengthReducer(leftWord, rightWord):
    if len(leftWord) > len(rightWord):
        return leftWord
    else:
        return rightWord

wrds.reduce(wordLengthReducer)

'great'

In [13]:
# count()

wrds.count()

7

In [14]:
# saveAsTextFile()
# Сохраним наши слова в виде "файла" - как обычно для Spark: будет создана директория,
# в которую будет записан RDD, каждый раздел - отдельный файл. Количество разделов мы тоже выведем,
# хотя это - 3 (мы именно так создавали RDD - см. выше).

!rm -rf words.txt
wrds.saveAsTextFile("words.txt")
wrds.getNumPartitions()

3

In [15]:
numbs = spark.sparkContext.parallelize([1.0, 5.0, 43.0, 10.0])
print(numbs.max()," as numbers")
print(numbs.max(key=str)," as strings")

43.0  as numbers
5.0  as strings


In [20]:
# трансформация glom()
# Собирает все элементы раздела в список.

print(wrds.collect())
print(wrds.glom().collect())

['this', 'is', 'spark', 'and', 'it', 'is', 'great']
[['this', 'is'], ['spark', 'and'], ['it', 'is', 'great']]


In [23]:
# Кэширование
# Как и с датафреймами - не сможем это эффективно использовать, но упомянуть стоит.
# cache() - кэширует RDD (после выполнения любого следующего действия), после этого RDD уже не будет
# каждый раз исполняться, значения будут браться из кэша.
# После выполнения этого узла можно будет увидеть статус RDD здесь - localhost:4040/storage/

wrds.cache()
wrds.count()

7

In [24]:
# Обратная операция - unpersist()
# После ее выполнения кэш пропадет (см. localhost:4040/storage/)

wrds.unpersist()

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:262

In [25]:
# Использование RDD для небольших XML файлов
# Как один из вариантов практического использования RDD - работа с мелкими XML файлами
# (помните, мы разбирали проблематику в модуле про работу с источниками?).
# Можно считать XML (в преобразованном, конечно, виде) в RDD, а дальше с помощью map() "вытащить"
# нужные поля в "плоскую" часть, оставив весь исходный XML в виде одного из полей.
# Потом эту "регулярную" структуру сохранить в реляционной таблице, например.
# Это будет не очень эффективно - см выше - но для обработки относительно небольшого потока XML может и хватить.

dList = [ 
    { "id": 1, "details": { "name": "pete", "phone": 123 } },
    { "id": 2, "details": { "name": "mike", "phone": 999 } },
]
dictRdd = spark.sparkContext.parallelize(dList)

def mkRecord(el):
    return ( el["id"], el["details"]["name"], el)

dictRdd.map(mkRecord).collect()

[(1, 'pete', {'id': 1, 'details': {'name': 'pete', 'phone': 123}}),
 (2, 'mike', {'id': 2, 'details': {'name': 'mike', 'phone': 999}})]

###  Пример вычисления числа Пи
Давайте, наконец, разберем - что же происходило в том примере, который мы использовали для вычисления числа Пи (в начале - когда тестировали работоспособность Spark), код приведен ниже. Теперь мы все знаем и можем объяснить - собственно, один из "смыслов" изучения RDD:

- создается простейщий RDD (один раздел, список чисел нужного диапазона)
- функция filter() позволяет нам выполнить код на питоне (вообще говоря - любой), в этом примере параметр (элемент RDD, для которого происходит вызов) не используется
- функция inside() написана на python, использует его инструментарий
- возвращает true или false, в зависимости от того, попали ли случайные числа в круг диаметром 2 или нет
- фильтруя мы оставляем в RDD только "попавшие" в круг его элементы
- count() посчитает число попавших в круг элементов
- далее - простая математика (которая геометрия)

Все просто! ("как хорошо уметь читать...")

In [31]:
import random
num_samples = 1000000

def inside(p):     
  x, y = random.random(), random.random()
  return x*x + y*y < 1

count = spark.sparkContext.parallelize(range(0, num_samples)).filter(inside).count()

pi = 4 * count / num_samples
print(pi)

3.141632


In [ ]:
spark.stop()